<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/product_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


#Import

In [2]:
!pip install PTable

In [3]:
!pip install tabulate

In [4]:
#For the system
import os
from prettytable import PrettyTable, HEADER
from tabulate import tabulate

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time

#Manage of files
import pandas as pd
import csv

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

# Tools

## Centered Len

In [5]:
def centered_len(max_len):
    def centered_string(string):
        if type(string) != 'string':
            string = str(string)
            
        rest = max_len - len(string)
        n_spaces = int(rest/2)

        string_spaces = ' ' * n_spaces
        
        centered_string = string_spaces + string + string_spaces

        if len(centered_string)<max_len:
            centered_string = ' ' + centered_string

        return centered_string
    return centered_string

## Count IF

In [6]:
def count_if(array, value):
    count = 0
    for element in array:
        if element == value:
            count += 1
    return count

In [7]:
%cd 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing'
!ls

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
'Day Update'	 Masters   Products
'Edition Test'	 Origin   'Untitled document.gdoc'


In [9]:
countries = ['mx', 'br']

In [11]:
# scraps_dir = f'Masters/{country}/parquet'
# products_dir = f'Products/{country}/parquet'

In [12]:
# table = PrettyTable()
# table.border = False
# table.FRAME = HEADER

In [10]:
category = 'music'
country = 'mx'

scraps_dir = f'Masters/{country}/parquet'
products_dir = f'Products/{country}/parquet'

scrap_file = f'{scraps_dir}/{country}-master_db_{category}.parquet'

In [13]:
def column_to_set(df, df_column, remove_None=True, sort=False, raw_data=False):
    raw_info = df[df_column].tolist()
    #Remember, this gets de None Value
    array = list(set(raw_info))

    if remove_None == True:
        None_cases = count_if(raw_info, None)
        if None_cases:
            array.remove(None)

    if sort == True:
        array = sorted(array)

    if raw_data == True:
        return array, raw_info
    
    else:
        return array

In [20]:
df = pd.read_parquet(scrap_file)

time_array = df['time'].to_list()
ranks = df['Rank'].to_list()
product_names = df['Product Names'].to_list()
product_IDs = df['Product ID'].to_list()
images_id = df['Image ID'].to_list()
stars = df['Stars'].to_list()
reviews = df['Reviews'].to_list()
authors = df['Authors/Company'].to_list()
editions = df['Edition/Console'].to_list()
prices_1 = df['Price_std_or_min'].to_list()
max_prices = df['Max_prices'].to_list()

In [16]:
len(product_IDs)

17750

In [17]:
df['Product Names']

0                                             FINE LINE
1                             GO LIVE (LIMITED VERSION)
2     Chromatica - Empaque de pasta dura- Incluye: C...
3                     The Dark Side Of The Moon (Vinyl)
4                                    Map Of The Soul: 7
                            ...                        
45                                Getz/Gilberto (Vinyl)
46    Euphoria (Original Score From the HBO Series) ...
47                                 Bocanada [Importado]
48                   Cheek to Cheek (Vinyl) [Importado]
49                     Sigh No More (Vinyl) [Importado]
Name: Product Names, Length: 17750, dtype: object

In [30]:


products_list = column_to_set(df, 'Product Names', sort=False)
start = (len(product_column) - 1)

product_ID = [None] * len(products_list)
product_url = [None] * len(products_list)
product_rank_image = [None] * len(products_list)

for product in range(len(products_list)):
    product_name = products_list[product]
    check = start

    # print(len(product_column), check)
    while product_column[check] != product_name:
        check -= 1
        if check < 0:
            print(len(product_column), check, product, product_name)
            break

    product_ID[product] = product_IDs[check]
    product_url[product] = f'https://www.amazon.com.{country}/dp/{product_IDs[check]}'
    product_rank_image[product] = f'https://images-na.ssl-images-amazon.com/images/I/{images_id[check]}.jpg'
    # print(product, product_name, product_ID[product], product_url[product] )

headers = ['Product ID', 'Product Name', 'Image ID (ranking page)', 'Image ID (product page)',
           'Last Update', 'Rank', 'Stars','Reviews', 'Price (Min or Mean)', 'Max Price', 'Last Review', 'Description']
info = {headers[0] : products_list,
        headers[1] : product_ID,
        headers[2] : product_url,
        headers[3] : product_rank_image}

print(tabulate(info, headers=headers), end='\n\n')

Product ID                                                                                                                   Product Name    Image ID (ranking page)                  Image ID (product page)
---------------------------------------------------------------------------------------------------------------------------  --------------  ---------------------------------------  -----------------------------------------------------------------------------------------
Ahora                                                                                                                        B07QL3XC41      https://www.amazon.com.mx/dp/B07QL3XC41  https://images-na.ssl-images-amazon.com/images/I/71prRZzM8CL._AC_UL200_SR200,200_.jpg
Paradigm                                                                                                                     B07SZBK3XH      https://www.amazon.com.mx/dp/B07SZBK3XH  https://images-na.ssl-images-amazon.com/images/I/71yVfmGk8RL._AC_UL200_S

In [ ]:
headers = ['Product ID', 'Product Name', 'Image ID (ranking page)', 'Image ID (product page)', 'Last Update', 'Rank', 'Stars','Reviews', 'Price (Min or Mean)', 'Max Price', 'Last Review', 'Description']
print(len(headers))

# dbs = { headers[0]: cont,
#         headers[1]: databases}

# print(tabulate(dbs, headers=headers), end='\n\n')
